In [6]:
from openai import OpenAI
orgID = %env CHAT_GPT_ORGANIZATION_ID
api_key = %env CHAT_GPT_API_KEY

client = OpenAI(
    api_key = api_key,
    organization=orgID,
)

from googleapiclient.discovery import build
import json
import re
from isodate import parse_duration
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd

api_key = %env YOUTUBE_SCAP_KEY
youtube = build('youtube', 'v3', developerKey=api_key)



In [44]:
channel_id = 'UCcefcZRL2oaA_uBNeo5UOWg' #ycombinator
video_id = '6lY9CYIY4pQ'



def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return transcript
    except Exception as e:
        print(f"An error occurred: {e}")
        return ''


tran = get_transcript(video_id)


justText = ''
textAndTimestamps = ''
for text in tran:
    textAndTimestamps += str(int(text['start'])) + '\n' + text['text'] + '\n'
    justText += text['text'] + '\n'



In [53]:
node_prefix = 18
node_start_sym = 'a'


topics = [{
	"start": 78,
	"finish": 145  
},{
	"start": 145,
	"finish": 230
},{
	"start": 230,
	"finish": 443
},{
	"start": 443,
	"finish": 512
},{
	"start": 512,
	"finish": 991 
},{
	"start": 991,
	"finish": 1109 
},{
	"start": 1109,
	"finish": 1298
},{
	"start": 1298,
	"finish": 1545
},{
	"start": 1545,
	"finish": 1697 
},{
	"start": 1697,
	"finish": 1822 
},{
	"start": 1822,
	"finish": 1993  
},{
	"start": 1993,
	"finish": 2265
},{
	"start": 2265,
	"finish": 2305 
}]

In [51]:
import os

startNum = node_prefix
startChar = node_start_sym
path = '/Users/shota/github/content-ge/startup-school/nodes'


def make_folder(name):
    newNode = os.path.join(path, name)
    if not os.path.exists(newNode):
        os.makedirs(newNode)
    newFile = os.path.join(newNode, "text1.md")
    f = open(newFile, 'w')
    f.close()
    


for topic in topics:
    name = str(startNum) + startChar
    print('"' + name + '": ' + json.dumps(getTopic(topic, video_id), indent=4) + ',')
    make_folder(name)
    startChar = chr(ord(startChar) + 1)


"18a": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "6lY9CYIY4pQ",
            "start": 78,
            "end": 145
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
"18b": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "6lY9CYIY4pQ",
            "start": 145,
            "end": 230
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
"18c": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    ],
    "video_parts": [
        {
            "id": "6lY9CYIY4pQ",
            "start": 230,
            "end": 443
        }
    ],
    "node_type": "lecture",
    "parent_id": "startup-school",
    "thumbnail": ""
},
"18d": {
    "name": "",
    "desctiption": "",
    "texts": [
        "text1.md"
    

In [ ]:
systemsText = """
You are a friendly and approachable expert in the startup domain, catering to a wide audience including those with varying levels of knowledge and experience. While maintaining a professional demeanor, your interactions should be warm and engaging, making complex startup concepts accessible and less intimidating. Stick to knowledge that is typically taught in educational settings, unless specifically asked to delve into more advanced or specialized topics. When providing answers, use simple language, prioritizing the 2000 most common words to ensure clarity and ease of understanding. This approach will make your advice more relatable and easier to grasp, especially for beginners. In situations requiring more advanced terminology or concepts not commonly taught in schools, clarify and simplify these terms as much as possible. Your primary goal is to educate and inspire aspiring entrepreneurs in a friendly and accessible manner, making the world of startups approachable for everyone.

You will be presented with the transcipt of a topic. Your goal is to produce:
1. Name of the topic
2. Very very small description of the topic
3. Rewrite this topic from your own perspective, user simple words, use markdown to highlight. Be the best 1-on-1 tutor
"""

def overlaps(a1, b1, a2, b2):
    int1 = pd.Interval(a1, b1)
    int2 = pd.Interval(a2, b2)
    return int1.overlaps(int2)

def getTranscriptForTopic(topic, transcript):
    start = topic['start']
    end = topic['finish']
    text = ''
    for snip in transcript:
        if overlaps(start, end, snip['start'], snip['start'] + snip['duration']):
            text+=snip['text']+' '
    return text

def getTopic(topic, video_id):
    return {
        "name": "",
        "desctiption": "",
        "texts": ["text1.md"],
        "video_parts": [{
            "id": video_id,
            "start": topic["start"],
            "end": topic["finish"]
        }],
        "node_type": "lecture",
        "parent_id": "startup-school",
        "thumbnail": ""
    }


startNum = node_prefix
startChar = node_start_sym

for topic in topics:
    t = getTranscriptForTopic(topic, tran)
    resp = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
              "role": "system",
                "content": systemsText
            },
            {
              "role": "user",
                "content": t
            }
        ],)
    print("====================================")
    name = str(startNum) + startChar
    newFile = os.path.join(os.path.join(path, name), "text1.md")
    f = open(newFile, 'w')
    print(resp.choices[0].message.content)
    f.write(resp.choices[0].message.content)
    f.close()
    startChar = chr(ord(startChar) + 1)
    print("====================================")

1. Topic Name: 
   Startup Challenges: Product-Market Fit and Customer Acquisition

2. Very Very Small Description: 
   Understanding the challenges faced by startups in achieving product-market fit and acquiring customers.

3. Rewritten Topic:

Here's a fact I've come across a lot in my journey: The majority of startups are on a tightrope struggling to find a balance - this is where they're trying to achieve something called **product-market fit**. They have high hopes, believing they've got it all figured out, but in reality, things aren't always rosy. So, if you're a startup founder, it's very likely you'll find yourself on this tightrope too.

Now, there's this common thought among newbie startup people, they think, "If I make my product and show it to the world, bingo - it'll be a success!" But let's come back down to earth. Sadly, that's not how it usually works. Our world is chock full of hustle and bustle, and let me tell you, there's no crowd waiting eagerly with open arms for

1. Topic Name: Understanding Product-Market Fit
2. Very Very Small Description: Understanding how to measure Product-Market Fit and identify the key metrics that signify a startup's alignment with its target market's needs.
3. Rewrite:

Alrighty! Today, we're going to talk about something you might have heard quite often in startup circles - product-market fit. Many find it hard to measure and understand clearly which often leads to false beliefs of having achieved it. Sounds familiar? No worries, let's break it down.

At the very core, product-market fit means you have created a product that people actually want. It's like saying, "I made something great, so now I've got product-market fit." We need to learn how to measure it and understand when we've hit the mark. 

**The best way to measure Product-Market Fit? Data!** Unbiased data is the key here! Here are a couple of steps you can consider:
1. **Identify your key metric**: This is the data point that best represents the value your